<a href="https://colab.research.google.com/github/cras-lab/Finance/blob/main/Beta_and_Volatility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

KOSPI200의 시가총액 상위 10개 종목의 시장베타와 변동성을 알아본다.

먼저 필요한 모듈을 임포트 한다. 선형회귀를 위해 stats도 임포트 한다.

In [44]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy import stats

KOSPI200 상위 10개 종목의 코드와 이름을 리스트로 만든다. <BR>
마지막은 KOSPI 지수를 읽어와야 하므로, 항목은 모두 11개가 된다.

In [45]:
names = ["Samsung", "LG Ensol", "SK hynix", "SS Bio", "LG Chem", "SS SDI", "HD Motors", "KIA", "POSCO Holding","NAVER", "^KS11"]
tickers = [ "005930.KS",  "373220.KS", "000660.KS", "207940.KS", "051910.KS", "006400.KS", "005380.KS", "000270.KS", "005490.KS",  "035420.KS" , "^KS11"]

10개 종목의 베타와 변동성을 저장할 배열을 선언한다.<BR>
베타는 slope에 변동성(표준편차)은 std에 저장한다.

In [46]:
slope = np.zeros( len(names) - 1 )
std = np.zeros( len(names) - 1 )

종목별 주가의 조정종가만 읽어 온다.

In [47]:
df = yf.download( tickers, "2022-01-01", "2022-12-31", progress=False)['Adj Close']

식별이 쉽도록 코드로 된 열대신 모두 종목 이름(names)으로 열을 대체한다.

In [48]:
df.columns = names

먼저 월별 수익률을 계산하고 이를 ret라는 데이터프레임에 저장한다.

In [49]:
ret = df.resample('M').apply(lambda x: (x[-1] - x[0])/x[0]).ffill(axis=1)

루프 돌면서 선형회귀를 통해 베타값을 찾고, 동시에 월 평균 변동성도 계산한다.<BR>
Ri = b + beta*Rm + error (즉 Y = ax+ b에서 각 자산은 Y, 시장은 X)

In [50]:
for i in  range(0, len(tickers)-1):
  slope[i], intercept, r_value, p_value, std_err = stats.linregress(ret[names[-1]], ret[names[i]])
  std[i] = df[names[i]].pct_change().ffill().resample('M').std().mean()

이제 도식화 해 본다.